In [2]:
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

In [22]:
# f(x) = 4x^2+6.8x-10.13
np.random.seed(0)

n = 1000
# Xs should be between -1 and 1, or apply norm
x=np.random.uniform(-1,1,n).reshape(-1, 1)
x1=x**2

print(x.shape, x1.shape)
X=np.concatenate((x1,x,np.ones((n,1))),axis=1)
coefs = np.array([-4, 6.8, -10.13])
y=X @ coefs
print(x.shape, y.shape)
fig=go.Figure()
fig.add_trace(go.Scatter(x=x.reshape(-1),y=y, mode='markers'))
fig.show()

(1000, 1) (1000, 1)
(1000, 1) (1000,)


In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.1,random_state=0)

print(X_train.shape, X_test.shape, X_val.shape)

(630, 3) (300, 3) (70, 3)


In [6]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=X_train[:, 1],y=y_train, mode='markers', name='Train'))
fig.add_trace(go.Scatter(x=X_test[:, 1],y=y_test, mode='markers', name='Test'))
fig.add_trace(go.Scatter(x=X_val[:, 1],y=y_val, mode='markers', name='Val'))
fig.show()

In [20]:
def MSE(y_true, y_pred):
    """
    Calculates the mean squared error

    Args:
        y_true (np.array): Target variable
        y_pred (np.array): Target Prediction 

    Returns:
        float: Mean sqaured error
    """
    return np.sum((y_true-y_pred)**2)/y_true.shape[0]

def forward(x,theta):
    return np.dot(x, theta)

def min_max_norm(x):
    return (x - x.min())/(x.max() - x.min())

def z_index_norm(x):
    return (x - x.mean())/(x.std())


In [15]:
def gradient_descent(X_train, X_test, X_val, y_train, y_test, y_val, lr=0.001, num_iter=1000, eps=10e-4, verbose=1, print_per=100):
    """Trains a polynomial regresion model, and evaluates on test and validation data.

    Args:
        X_train (np.array): Train dataset
        X_test (np.array): Test dataset
        X_val (np.array): Validation dataset
        y_train (np.array): Target train variable
        y_test (np.array): Target test variable
        y_val (np.array): Target validation dataset

    Returns:
        np.array: Parameters of regression model.
    """
    # theta = np.random.rand(X_train.shape[1])
    theta = np.zeros(X_train.shape[1])
    best_theta = np.copy(theta)
    h_test = forward(X_test, theta)
    error_test = MSE(y_test, h_test)
    min_error = error_test
    if verbose >= 1:
        print(f'Initial error on test is {error_test}')
    iter = 0

    while np.abs(error_test) >= eps and iter < num_iter:
        iter += 1
        h_train=forward(X_train,theta)
        J=MSE(y_train,h_train)
        for i in range(X_train.shape[1]):
            dtheta = -2/X_train.shape[0]*np.sum(np.dot(y_train-h_train,X_train[:,i]))
            theta[i]-=lr*dtheta

        h_test = forward(X_test, theta)
        error_test = MSE(y_test, h_test)

        if iter%print_per == 0 and verbose >= 1:
            print(f'Iter: {iter}/{num_iter}, train error: {J}, test_error: {error_test}')
        if error_test < min_error:
            min_error = error_test
            best_theta = np.copy(theta)

    h_val = forward(X_val, theta)
    error_val = MSE(y_val, h_val)

    if verbose >= 1:
        print(f'Error on validation: {error_val}')

    return best_theta
    

In [16]:
theta = gradient_descent(X_train, X_test, X_val, y_train, y_test, y_val, print_per=1000, num_iter=10000)
theta, coefs

Initial error on test is 157.38300819772414
Iter: 1000/10000, train error: 5.632360634168793, test_error: 6.207327986260238
Iter: 2000/10000, train error: 1.1011667114154458, test_error: 1.1580115239764692
Iter: 3000/10000, train error: 0.28598076566680053, test_error: 0.2978280432729476
Iter: 4000/10000, train error: 0.07745677895707628, test_error: 0.08137017557756117
Iter: 5000/10000, train error: 0.02254259497568563, test_error: 0.02409246402301471
Iter: 6000/10000, train error: 0.007644466436309369, test_error: 0.008328246681690004
Iter: 7000/10000, train error: 0.0032971252675959594, test_error: 0.0036288243503599317
Iter: 8000/10000, train error: 0.001815493568920965, test_error: 0.0019916315927013713
Iter: 9000/10000, train error: 0.0011723252710237008, test_error: 0.0012739612066807386
Error on validation: 0.0009367587881587397


(array([ -3.90037731,   6.78647422, -10.16635532]),
 array([ -4.  ,   6.8 , -10.13]))